In [2]:
import numpy as np
data = np.genfromtxt("data/data.csv",delimiter=',',skip_header=1)

In [31]:
def norm_mileage(data):
    mileage_mean = np.mean(data[:,0])
    mileage_std = np.std(data[:,0])
    new_data = np.column_stack(((data[:,0]-mileage_mean)/mileage_std,data[:,1]))
    return(new_data,mileage_mean,mileage_std)

def init_theta():
    return (0,0)

def update_theta(theta_old,data_normed,learning_rate):
    prices = data_normed[:,1]
    mileages= data_normed[:,0]
    estimates = theta_old[0]+theta_old[1]*mileages
    m = prices.size
    theta0_grad = 2*np.mean(np.add(estimates,-prices))
    theta1_grad = 2*np.mean(np.multiply(np.add(estimates,-prices),mileages))
    return (theta_old[0]-learning_rate*theta0_grad,theta_old[1]-learning_rate*theta1_grad)

def correct_theta(theta,mileage_mean,mileage_std):
    return(theta[0]-mileage_mean/mileage_std,theta[1]/mileage_std)

def train_model(data):
    (data_normed,mileage_mean,mileage_std) = norm_mileage(data)
    theta = init_theta()
    for i in range(0,1000):
        theta = update_theta(theta,data_normed,0.01)
    theta = correct_theta(theta,mileage_mean,mileage_std)
    return(theta)

In [32]:
train_model(data)

(6329.873352197463, -0.021448963555604397)